# Markov chains 

In this exercise we are going to look at a 3-state Markov chain with the following transition matrix:

$$
\mathbf{A} = \left(
\begin{matrix}
0.3 & 0.5 & 0.2 \\
0.4 & 0.3 & 0.3 \\
0.2 & 0.5 & 0.3 
\end{matrix}
\right)
$$

Before starting please take some time to do the following things in your note:

- Draw the transition graph for this Markov chain
- Identify which states are transient and whcih states are recurrent in this Markov chain.
- Look up the Chapmann-Kolmogorov (https://www.youtube.com/watch?v=W5P4kCpdhho) relation and explain what it means in your notes.

Once you have completed these tasks then move on to the remainder of the exercises contained herein. As always these tasks begin with some code for dynamic plotting that I have written for you so please press shift and enter on the cell below to begin.

In [ ]:
import math
%matplotlib notebook
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib import rc
from IPython.display import HTML
import random
rc('animation', html='none')

class plotobj(object):
    def __init__(self,ny,xmin,xmax):
        self.fig = plt.figure()
        self.ny = ny
        self.xmin = xmin
        self.xmax = xmax
        self.ax = plt.axes(xlim=(xmin,xmax), ylim=(0, 1))
    
    def setup(self):
        self.fno = 0
        xdata,ydata = [],[]
        for i in range(self.xmin,self.xmax):
            xdata.append(i)
        for j in range(0,self.ny):
            ydata.append([])
            for i in range(self.xmin,self.xmax):
                ydata[j].append(0)
    
        self.rects = []
        collist = ['r','g','b']
        for j in range(0,self.ny):
            self.rects.append( self.ax.bar(xdata,ydata[j], 0.1, color=collist[j] ) )
            for k in range(self.xmin,self.xmax) :
                xdata[k] += 0.1
                
    def run(self,data):
        xdata,ydata=data
        self.fno +=1
        self.fig.suptitle('frame number ' + str(self.fno), fontsize=14, fontweight='bold' )
        maxy,j=0,0
        for rbunch in self.rects :
            i=0
            for rect in rbunch :
                if self.ny==1 :
                    rect.set_height(ydata[i])
                    if( ydata[i] > maxy ) :
                        maxy = ydata[i]
                else :
                    rect.set_height(ydata[j,i])
                    if( ydata[j,i]>maxy ) :
                        maxy = ydata[j,i]
                i+=1
            j += 1
        self.ax.set_ylim(0,maxy+0.1)
        return self.rects

def dynamicplot( ny, ngen, operation, xmin, xmax, myvar, tinverval=1):
    myplot = plotobj(  ny, xmin, xmax )
    return anim.FuncAnimation(fig=myplot.fig, func=myplot.run, frames=operation( ngen, myvar ), 
                                init_func=myplot.setup, interval=10, blit=False, repeat=False)




# Step 1: Manipulating matrices using python

The first thing we need to do in the exercise involves learning how python can be used to manipulate matrices.  We are eventually going to perform simulations of the Markov chain and to perform these simulations correctly we are going to need the information within the transition matrix to do things correctly.  Furthermore, it is useful to keep the structure of the matrix.  By now you have learnt many things about matrices so it better to use this familiar mathematical structure than to invent a new structure.  

In python there is a type of object called an array that can be used to store matrix.  Essentially an array is a list of lists the transition matrix $A$ above is stored as a 3 lists each of which contains three numbers.  We create this array object as follows:

In [ ]:
A = np.array( ((0.3,0.5,0.2),
               (0.4,0.3,0.3),
               (0.2,0.5,0.3) ))

This is hopefully not too difficult to understand as the way I have written it here is similar to the way that I wrote it in the first equation with the numbers in rows and columns.  To be clear python does not require me to write it this way.  I can equally write it all on one line like this:

In [ ]:
A = np.array( ((0.3,0.5,0.2), (0.4,0.3,0.3), (0.2,0.5,0.3)) )

What we are using here is an object (or class) that is part of a python package known as numpy (which I have renamed np).  In other words, we are using the object oriented paradigm again (see this video ).  When we call np.array we are thus calling the constructor for the array object and making it so that if we use the variable A in the future this refers to an instance of the array object.  If you remember from the exercise on calculating histograms when we call the constructor we create an instance of a class.  The constructor must therefore and set all the internal variables in the class, which is what the above command does.  

After the np.array you will notice that three sets of brakets are opened.  The first of these sets of brackets closes at the end of the statement.  Between this set of brackets are all the arguments that are passed to the function.  In other words, this set of brackets just the standard set of brackets that surround the arguments to a function as we have see before on many occasions.  The second set of brackets starts a list object - we have seen lists before as well in programs like this one:

In [ ]:
myl = (0,1,2,3)
print( myl[0], myl[2] )

The difference between the program above the list we created in the above program and the list we are creating when we construct our array is what the elements of the list are.  In the program above the elements of the list are numbers.  Our list can contain anything though.  For example it can contain numbers, strings, or even another list.  This ability to fill lists with anythign is feature of some programming languages that is known as static polymorphism.  

When creating our matrix we thus use a list of lists.  We thus have an outer list that has  three elements within it and each of these elements is a list of three numbers.  To test whether you have understood this concept look at the program below.  When this is run it will output three lines of text.   Write out what you think it will output on each line before you run it.  Then run it and see if your prediction was correct.

In [ ]:
wordlist = (("lists","can","contain"), ("everything","including"),("lists"))

# This loop runs over the items in the list 
# wordlist.  To understand what the program will 
# output you thus have to understand what items
# are contained in the elements of wordlist
for word in wordlist :
    print( word )

Hopefully you now understand what was passed to the array constructor.  Essentially we have three lists each of which contains the elements from one of the rows of the matrix.  Easy enough.

Now notice that we can access elements of the matrix using a rather obvious syntax:

In [ ]:
print( A[0,0], A[0,1], A[1,1], A[2,1], A[2,2] )

In other words the command A[0,1] is used when we to do something to the element in the 0th row and the 1st column of the matrix.  We can thus think of any expression with the set of symbols A[0,1] as some expression that involves the variable in the 0th row and the 1st column of matrix.  This is the same way that we work with matrices when we write mathematical equations so you shouldn't be struggling to much with this idea. 

To see if you have understood see if you finish the function below that takes two 3x3 matrices as input and that returns the matrix product.  Hint: To write this function you will have to write three nested for loops: 

In [ ]:
def product( A, B ) :
    C = np.array( ((0.,0.,0.),(0.,0.,0.),(0.,0.,0.)) )
    return C

Use the function you have written to calculate the square of the matrix A.    

Notice that there is a function (np.dot) within numpy to calculate matrix products for you.  In the cell below I have used this function to calculate the square of A for you so you can compare the answer you get when you press shift and enter on the cell below we the answer that you get from your function to see if you got things right.  

In [ ]:
print( np.dot( A, A ) )

# Simulating the Markov chain

We are going to come back to working with matrices later in this exercise in this next part though we are going to learn how to simulate a random process that can be described using the Markov chain.  What we are going to do is monitor the fraction of time that is spent in each of the three states of the Markov chain when we run the Markov chain with the transition matrix given in the previous section.  We are thus going to begin by introducing a slightly different variant on the histogram code that I introduced in the exercise on constructing histograms:

In [ ]:
class histoclass(object) :
    def __init__( self, xmin, nstates, initstate ):
        self.xmin=xmin
        self.initstate=initstate
        self.xdata = []
        self.ycount = []
        for i in range(0,nstates) :
            self.xdata.append( xmin + i )
            self.ycount.append(0)
            
    def buildHistogram( self, ngen, myvar ) :
        cnt=0
        pstate = self.initstate    
        while cnt<ngen :
            cnt += 1
            # Notice that the random number generator function now 
            # depends on the value generated for the previous random variable 
            # This is the Markov property.
            X = myvar( pstate )   
            pstate = X
            nbin = X - self.xmin
            self.ycount[nbin] += 1
            tdata = []
            for dat in self.ycount :
                tdata.append( dat / cnt )
            yield self.xdata, tdata

The most important thing to note in the above code is that the random number generator (myvar) now takes in an argument.  The argument this takes in is the value the random variable had during the previous step.  This key difference between this code and the codes that we have used in previous exercises is the Markov property in action.  Lets now turn to the random number generator itself.  The code for this part is in the cell below:

In [ ]:
# Notice the random variable here depends on an 
# argument (instate) this is the current state that
# we are in.
def genmove( instate ):
    # Generate a uniform random variable between 0 and 1
    rr = random.uniform( 0, 1 )
    # If the uniform random variable is less than the transition
    # probability from instate to state 0 we move to state 0
    if rr<A[instate,0]  :
        return 0
    # If the uniform random variable has a value between the transition
    # probability for moving between state 0 and state 0 and if it is less
    # that the total probability of moving to either state 0 or state 1 we
    # move to state 1
    elif rr<(A[instate,0] + A[instate,1]) : # NB Python uses elif as a short form of else if
        return 1
    # If the uniform random variable is greater than the total probability of
    # moving to either state 0 or state 1 we move to state 2.
    return 2

Lets take a look at how the code above works.  We first generate a uniform random variable that lies between 0 and 1.  We then check whether this number is less than the variable in the instate'th row and 0th column of our transition matrix A.  This element of the matrix gives the probability of a transition between the instate'th state and state 0.  If our random variable is less than this probability then we move to state 0.  If it is greater than this probability but less than the total probability of moving to state 0 or state 1 then we move to state 1.  If it is greater than this total probability then we move to state 2.  The way this code works is illustrated in the figure below:

<img src="Images-ergodic/gen-random-move.png" width="600">

We can in rewrite the above code using a for loop.  By doing so we can generalise the algorithm so that it works even if the Markov chain we are simulating has more than 3 states.  Try to see if you can work out how to write this general form of the algorithm.  Once you are done, or before, run the cell below which will simulate the chain for a while a generate a histogram showing the fraction of time spent in each of the states.  Describe the behavior of this histogram in your own words. How much do the heights of the columns change in the early part of the simulation and how much do they change in later parts.   

In [ ]:
# This creates an instance of the histogram class
# the arguments are the index for the first state (0),
# the total number of states in the chain (3) and the
# state that we are initially in (1)
myhisto = histoclass( 0, 3, 1 )
# Now generate a histogram the arguments here are
# The number of bar charts we are drawing - in this case just one - the probabilities
# The number of random numbers to generate = 100
# The function that calculates the data to plot = myhisto.buildHistogram
# Where to start (0) and end (3) the x axis
# The function that generates random variables = genmove
dynamicplot( 1, 100, myhisto.buildHistogram, 0, 3, genmove )

You will hopefully have noticed that that the heights of the columns initially fluctuate a lot.  However, after only a few steps the heights of the columns settle down to pretty constant values.  In other words, the fraction of time spent in each of the states of the chain settles down to a constant value.  This vector of probabilities is known as the limiting stationary distribution of the Markov chain and the reason that it emerges is explained in the following video: https://www.youtube.com/watch?v=JZnzQ8YzVZg

# The Chapman Kolmogorov Relation in practise

In the second part of this exercise we are going to look at the emergence of the limiting stationary distribution in Markov chains in a slightly different way.  In this part we are use the Chapman Kolmogorov relation (https://www.youtube.com/watch?v=W5P4kCpdhho) to calculate the $N$ step transition probabilities for our transition matrix.  If you remember the Chapman Kolmogorov relation tells us that the $(n+m)$-step transition matrix $\mathbf{A}^{n+m}$ is related to the $n$-step transition matrix $\mathbf{A}^{n}$ and the $m$-step transition matrix $\mathbf{A}^m$ by:

$$
\mathbf{A}^{n+m} = \mathbf{A}^{n} \mathbf{A}^{m}
$$

In other words, we can calculate the $n$-step transition matrix by taking the $n$th power of the 1-step transition matrix.  

In the cell below I have written a function that calculates these powers of the matrix and that then generates a movie showing how the values of theese transition probabilities change as the power increases.  In each frame of this movie three bar charts are shown in red, green and blue.  The red bars give the transition probabilities in the first row of the matrix, the green bars are the values of the transition probabilities in the second row of the matrix and the blue bars are the values of the transition probabiliies in the third row of the matrix.  Describe what happens to the heights of these bars as the powers of the matrix increases and what this implies for the chain. 

In [ ]:
class matrixpowers(object) :
    def __init__( self, nstates, tmatrix ):
        self.tmatrix = tmatrix
        self.xdata = []
        for i in range(0,nstates) :
            self.xdata.append( i )
            
    def calcPowers( self, ngen, myvar ) :
        cnt=0
        mpower = self.tmatrix    
        while cnt<ngen :
            cnt += 1
            # This multiplies the nth power of the transition
            # matrix by the transition matrix in order to generate
            # the (n+1)th power of the transition matrix
            mpower = np.dot( mpower, self.tmatrix )
            yield self.xdata, mpower 
            
powers = matrixpowers( 3, A )
dynamicplot( 3, 10, powers.calcPowers, 0, 3, genmove, 1000 )

What you should have noticed is that all the columns become equal after a relatively small number of steps.  In other words, the probability of moving from state 1 to state 2 over a 10 step interval is roughly the same as the probability of moving from state 0 to state 2 over a 10 step interval and the probability of moving from state 2 and back to state 2 over a 10 step interval.  If the number of steps taken is sufficiently large the initial state no longer matters and the probibility of being in state in the future can be thought of as a constant.  Once again, we see the emergence of the limiting stationary distribution but this time we have taken a slightly different route in getting here.  This route is explained in this video: https://www.youtube.com/watch?v=1v-GEdV8zys.

# Extracting eigenvalues and eigenvectors

The video you should have just watched (https://www.youtube.com/watch?v=1v-GEdV8zys) explains how the limiting stationary distribution is equal to the principle (left) eigenvector of the transition matrix and how eigenvalue corresponding eigenvalue to this eigenvector is equal to one.  In this final part we are going to learn how to extract the eigenvalues and eigenvectors of a matrix using tools within python and we are then going to confirm the results that were discussed in the video.

Lets begin by finding the eivenvalues and eigenvectors of the transition matrix, $\mathbf{A}$.  We can do this using python by exploiting the code below:

In [ ]:
# Notice that I calculate the eigenvalues of the transpose
# of the transition matrix here.  I do this because I want 
# to calculate the left eigenvectors.  In all disciplines 
# other than Markov chains we are interested in right 
# eigenvectors and not left eigenvectors.
vals, vecs = np.linalg.eig( A.transpose() )
print( "EIGENVALUES", vals )
lvecs = []
# This bit here ensures that the elements for each of my 
# eigenvectors all add up to one.
for i in range(0,3):
    lvecs.append([])
    norm = vecs[0,i] + vecs[1,i]  + vecs[2,i]
    lvecs[i].append( vecs[0,i]/norm )
    lvecs[i].append( vecs[1,i]/norm )
    lvecs[i].append( vecs[2,i]/norm )
    print("EIGVEC", i, vecs[0,i]/norm, vecs[1,i]/norm, vecs[2,i]/norm)
lvecs = np.array( lvecs )  


When you run the code above should find that the largest eigenvalue is equal to one in agreement with what was discussed in the video.  The eigenvector corresponding to this eigenvalue is the limiting stationary distribution.

Notice that I took the transpose of the matrix A before I diagonalised it.  I have to do this in order to calculate the left eigenvalues that I need in this case.  If you look at the top right eigenvalue, which can be calculated using the code below:

In [ ]:
vals, vecs = np.linalg.eig( A )
print( "EIGENVALUES", vals )
rvecs = []
for i in range(0,3):
    rvecs.append([])
    norm = vecs[0,i] + vecs[1,i]  + vecs[2,i]
    rvecs[i].append( vecs[0,i]/norm )
    rvecs[i].append( vecs[1,i]/norm )
    rvecs[i].append( vecs[2,i]/norm )
    print("EIGVEC", i, vecs[0,i]/norm, vecs[1,i]/norm, vecs[2,i]/norm)
rvecs = np.array( rvecs ).transpose()

you see that it is not very interesting.  Essentially all the components of this eigenvector are equal.

In the video (https://www.youtube.com/watch?v=1v-GEdV8zys) I explained how we can decompose square matrices using:

$$
\mathbf{A} = \mathbf{V} \Lambda \mathbf{V}^{-1}
$$

where $\mathbf{V}$ is a square matrix that contains the right eigenvectors in its columns, where $\mathbf{V}^{-1}$ is a square matrix that contains the left eigenvectors in its rows and where $\Lambda$ is a diagonal matrix that has the eigenvalues on the diagonal.  I then showed how the above decomposition allows us to write the $n$th power of the matrix as:

$$
\mathbf{A}^n = \mathbf{V} \Lambda^n \mathbf{V}^{-1}
$$

Last but not least I noted that when the largest eigenvalue is unique and equal to one we can approximate $\Lambda^n$ as:

$$
\Lambda^n \approx \left( 
\begin{matrix}
1 & 0 & 0 \\
0 & 0 & 0 \\
0 & 0 & 0
\end{matrix}
\right)
$$

because if $\lambda_1 > \lambda_2$ then $\left(\frac{\lambda_2}{\lambda_1}\right)^n \rightarrow 0$ as $n \rightarrow \infty$.  

In the cell below I thus reconstruct the matrix $\mathbf{A}^n$ using the eigenvectors that I calculated in the previous exercises and:

$$
\mathbf{A}^n = \mathbf{V} \Lambda^n \mathbf{V}^{-1}
$$

with $\Lambda^n$ set equal to:

$$
\Lambda^n \approx \left( 
\begin{matrix}
1 & 0 & 0 \\
0 & 0 & 0 \\
0 & 0 & 0
\end{matrix}
\right)
$$

in order to approximate the limiting behavior for the $n$-step transition matrix.  As you can see all the rows are equal, which once again explains how the limiting stationary distribution emerges.

In [ ]:
hp = np.array( ((1.,0.,0.),
                (0.,0.,0.),
                (0.,0.,0.)) )
# This bit does V \Lambda^n
prod1 = np.dot( rvecs, hp )
# Factor of three here just ensures that the rows sum to one and
# the np.dot finishes V \Lambda^n V^{-1}
print( 3*np.dot( prod1, lvecs ) )

# An exercise for you to try

The remainder of this exercise involves using what you have learnt by working through the previous sections to investigate the following Markov chains.

$$
\mathbf{A} = \left(
\begin{matrix}
0 & 1 \\
1 & 0
\end{matrix}
\right)
$$

As always start by drawing the transition graph and by identifying which states are transient and which are recurrent.  Once you have done this create a python array object that contains the transition matrix above.  You should then be able to use this array object to simulate the markov chain and to draw a histogram showing the fraction of time that is spent in each of the states.  You can then use the code from previous sections to look at the powers of the matrix and to draw a dynamic bar chart that shows how the transition probability matrix changes with time.  Lastly, calculate the eigenvalues and left eigenvectors of the matrix (you should be able to do this using only the code in the first box in the section above).  Describe the behavior you observe for your histogram, your matrix powers and the eigenvectors in your notes. Explain how the behavior you observe is in keeping with what you would expect to happen with a Markov chain of this type.